# Classification

Nosso principal objetivo na modelagem é realizar uma clusterização dos dados para posteriomente criar um sistema descritivo, ou seja, nossa meta é criar visualizações que possam ser úteis para os médicos.

Assim, mesmo que os nossos dados possuam rótulos devidamente prontos nós tentaremos utilizar a clusterização. Contudo, inicialmente iremos utilizar algoritmos supervisionados para averiguar se os dados na forma como estão conseguem distinguir bem as classes.

## 1. Leitura dos dados

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [2]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura dos dados
# df_parkinson = pd.read_csv('parkinson_normalizado_ss.csv',index_col='name')
# df_parkinson = pd.read_csv('parkinson_normalizado_mm.csv',index_col='name')
df_parkinson = pd.read_csv('parkinson_pca.csv',index_col='name')

#Mapeando dados para conseguir calcular o AUC
df_parkinson['drug'] = df_parkinson['drug'].map({'CBD':1,'placebo':0})

# 2. Modelagem

## 2.1 Classificação

Iremos realizar a modelagem com 3 algoritmos supervisionados:

- Árvore de Decisão
- Random Forest
- Regressão Logística

Como principal métrica de avaliaçã iremos utilizar o AUC (Área Under Curve) por nosso problema ser binário. De qualquer forma também iremos avaliar acurácia, sensibilidade, especificidade e erros para cada pessoa.

Por fim, para garantir o poder generativo dos nossos algoritmos será utilizado uma variação do k-fold.

Gostaríamos de lembrar que será utilizado os dados obtidos após a aplicação do PCA.

### 2.1.1 Sem variáveis categórias

In [17]:
'''Realiza uma avaliação da classificação'''
def evaluate_classifier(X,Y,split=1,type_clf='dt'):
    
    #Pessoas do dataset
    unique_people = X.index.unique()

    #Listas para armazenar os resultados finais
    list_score = []
    list_sens = []
    list_spec = []
    list_auc = []
    list_error = []

    for i in range(500):
        if(split == 1):
            #Sample das pessoas
            people_train = np.random.choice(unique_people,size=11,replace=False)
            people_test = unique_people[~unique_people.isin(people_train)]
    
            #Criando treino e test
            X_train = X.loc[people_train]
            Y_train = Y.loc[people_train]
            X_test = X.loc[people_test]
            Y_test = Y.loc[people_test]
        elif(split == 2):
            X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.4)

        #Criando classificador
        if(type_clf == 'dt'):
            clf = DecisionTreeClassifier().fit(X_train,Y_train)
        elif(type_clf == 'rf'):
            clf = RandomForestClassifier(n_estimators=100).fit(X_train,Y_train)
        else:
            clf = LogisticRegression(solver='liblinear',penalty='l1')

        #Matrix de confusao
        tn, fp, fn, tp = confusion_matrix(Y_test,clf.predict(X_test)).ravel()

        #Sensibilidade
        sens = tp/(tp+fn)
        
        #Especificidade
        spec = tn/(tn+fp)

        #Score
        score = (tn+tp)/(tn+tp+fn+fp)
        
        #AUC
        auc = roc_auc_score(Y_test,clf.predict(X_test))
        
        #Pessoas que erraram
        error = (clf.predict(X_test) == Y_test)
        list_error = np.append(list_error,error[error == False].index.unique().values)
        list_score.append(score)
        list_sens.append(sens)
        list_spec.append(spec)
        list_auc.append(auc)

    print('AUC:',np.nanmean(list_auc))
    print('Score:',np.nanmean(list_score))
    print('Sensibilidade:',np.nanmean(list_sens))
    print('Especificidade:',np.nanmean(list_spec),'\n')
    
    #Calculo do erro por pessoa
    error = pd.Series(list_error).value_counts()
    print(error/error.values.sum())

In [4]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float')
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y,split=2)

AUC: 0.518758064516129
Score: 0.518758064516129
Sensibilidade: 0.5169677419354839
Especificidade: 0.5205483870967742 

person_12    0.047304
person_11    0.046921
person_4     0.046634
person_21    0.046538
person_10    0.046347
person_8     0.046347
person_18    0.046347
person_16    0.046155
person_7     0.046155
person_20    0.045964
person_13    0.045964
person_17    0.045868
person_5     0.045772
person_19    0.045293
person_2     0.045293
person_0     0.045293
person_14    0.044815
person_1     0.044719
person_15    0.044240
person_3     0.043857
person_9     0.042133
person_6     0.042038
dtype: float64


### 2.1.2 Variáveis categórias sem ser dummy

In [5]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Adicionando variaveis categoricas
X['measure'] = LabelEncoder().fit_transform(df_parkinson['measure'])
X['evaluate'] = df_parkinson['evaluate']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4841038961038961
Score: 0.4841038961038961
Sensibilidade: 0.48207792207792205
Especificidade: 0.48612987012987013 

person_6     0.049663
person_7     0.049300
person_2     0.048208
person_14    0.047844
person_18    0.047844
person_3     0.047299
person_21    0.046935
person_10    0.046571
person_11    0.046389
person_9     0.046207
person_4     0.045843
person_0     0.045661
person_8     0.045479
person_17    0.044934
person_5     0.044388
person_20    0.043842
person_12    0.043478
person_1     0.043114
person_15    0.042569
person_13    0.042205
person_16    0.041841
person_19    0.040386
dtype: float64


### 2.1.3 Variáveis categórias sendo dummy

In [6]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'],prefix='measure'),pd.get_dummies(df_parkinson['evaluate'],prefix='evaluate')],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4828051948051948
Score: 0.4828051948051948
Sensibilidade: 0.4775844155844155
Especificidade: 0.488025974025974 

person_16    0.048217
person_10    0.048035
person_8     0.048035
person_5     0.048035
person_20    0.046943
person_13    0.046579
person_11    0.046397
person_0     0.046397
person_2     0.046215
person_9     0.046033
person_18    0.045852
person_1     0.045852
person_17    0.045852
person_19    0.045670
person_7     0.045124
person_4     0.044942
person_14    0.044578
person_12    0.044214
person_21    0.043486
person_6     0.041667
person_3     0.041667
person_15    0.040211
dtype: float64


### 2.1.3 Váriveis categóricas sendo dummy, porém unidas

In [7]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'] + '_' + df_parkinson['evaluate'].map(lambda x: str(x)))],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.501012987012987
Score: 0.501012987012987
Sensibilidade: 0.49955844155844153
Especificidade: 0.5024675324675325 

person_9     0.050373
person_7     0.048372
person_0     0.048191
person_1     0.047281
person_11    0.047099
person_16    0.046918
person_8     0.046554
person_19    0.046554
person_4     0.045645
person_21    0.045645
person_18    0.045463
person_6     0.044917
person_15    0.044735
person_12    0.044554
person_10    0.044554
person_14    0.044372
person_20    0.044190
person_2     0.044190
person_5     0.043644
person_17    0.042553
person_13    0.042371
person_3     0.041826
dtype: float64


A utilização de vários tipos de codificação para a variável fase e medida foram tentados por nossas análises mostrarem que existem diferenças dependendo da onde você olha.

Contudo, independente da forma de codificação utilizada não conseguimos bons resultados. Esse fato ocorreu mesmo havendo uma variação dos algoritmos supervisionados.

Acreditamos que essas codificações não derão certo por em cada fase/medida possuir um comportamente diferente. Vamos avaliar essa hipótese na próxima seção.

### 2.1.4 Classificação considerando apenas uma fase e medida

In [8]:
#Filtro selecionado a medida P na avaliação 1 com PC4 e PC5
df_parkinson = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 1),['PC4','PC5','drug']]

In [9]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.loc[:,['PC4','PC5']]
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.7476416666666666
Score: 0.7294545454545455
Sensibilidade: 0.7161269841269841
Especificidade: 0.7791563492063492 

person_8     0.133737
person_0     0.123656
person_15    0.088038
person_7     0.073253
person_11    0.069220
person_2     0.067876
person_17    0.064516
person_18    0.056452
person_6     0.054435
person_19    0.047715
person_9     0.040323
person_3     0.040323
person_1     0.033602
person_21    0.025538
person_16    0.022849
person_14    0.018145
person_12    0.015457
person_20    0.015457
person_5     0.002688
person_13    0.002688
person_10    0.002016
person_4     0.002016
dtype: float64


Podemos ver que agora obtivemos bons resultados. Contudo isso foi a custa de uma grande diminuição da quantidade de amostras no dataset.

Salientamos também que esse valor só foi possível de alcançar utilizando o PC4 e PC5. Qualquer outro componente utilizado acaba 'atrapalhando' na classificação.

### 2.1.5 Uso de novas features

Dado que não foi possível obter bons resultados com as features anteriores, iremos utilizar novos atributos para averiguar se conseguimos melhores valores.

As novas features calculadas foram baseadas em https://github.com/Luke3D/CIS_PD-NDM que fez um trabalho similar com dados de Parkinson.

In [2]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura
df_parkinson = pd.read_csv('parkinson_v2.csv',index_col = 'name')

#Apresentação de algumas linhas
df_parkinson.head()

,pse,pspf1,pspf2,pspf3,psp1,psp2,psp3,wpsf,RMS,range,...,var,skew,kurt,jerk_mean,jerk_std,jerk_skew,jerk_kur,evaluate,measure,drug
name,,,,,,,,,,,,,,,,,,,,,
person_0,3.021855,1,8,17,17243.324518,1850.713750,1693.279040,17243.324518,382.050996,-6848.679085,...,363.377887,1.412557,30.949340,0.050528,535.257543,-0.163219,30.336621,2,B,placebo
person_0,3.533303,1,3,6,39115.767987,13796.968242,5978.057257,39115.767987,390.853005,-3097.095783,...,908.940592,0.098040,4.969888,-0.922482,1153.757327,-0.175426,6.861516,2,P,placebo
person_0,0.859185,1,4,13,129866.558198,3193.568772,1810.657041,129866.558198,397.813776,-6028.623410,...,630.078604,0.073222,2.749642,-0.111951,814.505179,0.168532,3.834549,2,A,placebo
person_0,3.424263,1,4,7,102144.341817,60726.093359,42408.550846,102144.341817,470.023084,32858.801543,...,4939.505835,1.007991,0.771216,1.427418,6544.802604,0.251431,0.646722,2,S1,placebo
person_0,2.009236,1,3,5,150065.780097,50565.746519,23406.888141,150065.780097,477.519118,10429.318103,...,4692.390496,2.259779,6.046340,2.919689,6994.739267,-0.003559,3.055311,2,S2,placebo


In [21]:
df_parkinson.columns

Index(['pse', 'pspf1', 'pspf2', 'pspf3', 'psp1', 'psp2', 'psp3', 'wpsf1',
       'wpsf2', 'wpsf3', 'jerk_mean', 'jerk_std', 'jerk_skew', 'jerk_kur',
       'RMS', 'range', 'mean', 'var', 'skew', 'kurt', 'evaluate', 'measure',
       'drug'],
      dtype='object')

Explicação das novas features:

- jerk_(mean|std|skew|kur): Média, desvio padrão, assimetria e curtose para a série temporal com lag1.
- RMS: Root mean square da série temporal.
- range: Intervalo da série temporal.
- mean: Médida da série temporal.
- var: Variância da série temporal.
- skew: Assimetria da série temporal.
- kurt: Curtose da série temporal.

Vamos fazer algumas mudanças nos dados como criar novas colunas e aplicar a boxcox para normalizar.

In [22]:
#Mapenado droga como variavel numerica
df_parkinson['drug'] = LabelEncoder().fit_transform(df_parkinson['drug'])

#Criacao de novas colunas
df_parkinson['wpsf2'] = df_parkinson['pspf2'] * df_parkinson['psp2']
df_parkinson['wpsf3'] = df_parkinson['pspf3'] * df_parkinson['psp3']
df_parkinson.rename(columns={'wpsf':'wpsf1'},inplace=True)

#Arrumando posicao das colunas
cols = list(df_parkinson.columns[0:8]) + ['wpsf2','wpsf3','jerk_mean', 'jerk_std','jerk_skew', 'jerk_kur','RMS','range', 'mean', 'var', 'skew', 'kurt','evaluate','measure','drug']
df_parkinson = df_parkinson.loc[:,cols]

#Aplicando log
positive_cols = (df_parkinson <= 0).any()
positive_cols = positive_cols[~positive_cols].index
positive_cols = positive_cols.drop('evaluate')
df_parkinson.loc[:,positive_cols] = np.log(df_parkinson.loc[:,positive_cols])

#Reescalando variaveis
mm = MinMaxScaler().fit(df_parkinson.iloc[:,:-3])
df_parkinson.iloc[:,:-3] = mm.transform(df_parkinson.iloc[:,:-3])

Como possuímos 18 atríbutos, utilizararemos o <b>forward selection</b> para selecionar as melhores features.

In [23]:
#Arvore de decisao sera usada
dt = DecisionTreeClassifier()

#Criando foward selection
sfs1 = sfs(dt,k_features=3,forward=True,floating=False,scoring='roc_auc',cv=5)

#Divisao do dataset
X = df_parkinson.iloc[:,:-3]
Y = df_parkinson['drug']

#Selecao
sfs1 = sfs1.fit(X,Y)
print(sfs1.k_feature_names_)

#Classificacao
evaluate_classifier(X.loc[:,sfs1.k_feature_names_],Y)

('jerk_skew', 'RMS', 'mean')
AUC: 0.5238441558441558
Score: 0.5238441558441558
Sensibilidade: 0.5215064935064935
Especificidade: 0.5261818181818181 

person_0     0.048009
person_12    0.047827
person_11    0.047645
person_5     0.047281
person_13    0.046918
person_3     0.046736
person_1     0.046190
person_21    0.046190
person_18    0.046008
person_2     0.045827
person_16    0.045463
person_14    0.045281
person_17    0.045099
person_7     0.045099
person_6     0.044917
person_15    0.044554
person_8     0.044554
person_4     0.044372
person_9     0.044008
person_10    0.043281
person_19    0.042553
person_20    0.042189
dtype: float64


Em vários testes realizados manualmente, vimos que não foi possível obter bons resultados. 

Assim, utilizamos a mesma ideia anterior de realizar uma filtragem por fase/medida.

Com a ajuda do forward selection concluímos que para cada fase/medida deveríamos utilizar os seguintes atributos:

| Medida       | Fase     | Features     |
| :------------- | :----------: | -----------: |
| P | 1 | psp3, pspf3, jerk_mean|
| P | 2 | psp1, pspf1, jerk_mean|
| B | 1 | pspf3, wpsf3, jerk_std, range|
| B | 2 | pspf1, pspf3, var, jerk_std|
| A | 1 | Nenhum|
| A | 2 | wpsf2, kurt, jerk_kur|
| S1 | 1 | pse, kurt, jerk_mean|
| S1 | 2 | Nenhum|
| S2 | 1 | Nenhum|
| S2 | 2 | psp1, psp3|
| F1 | 1 | psp2, jerk_kur|
| F1 | 2 | pspf2, kurt, jerk_kur|
| F2 | 1 | pspf1, kurt|
| F2 | 2 | pspf1, pspf2|

Abaixo podemos ver uma demonstração.

In [29]:
# FILTRO - TESTE
df_test = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 2),['psp1','pspf1','jerk_mean','drug']]

#Separando dataset em variaveis  dependentes e independentes
X = df_test.select_dtypes('float')
Y = df_test['drug']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.7996698412698413
Score: 0.7870909090909091
Sensibilidade: 0.8097547619047618
Especificidade: 0.7895849206349206 

person_14    0.177626
person_0     0.105038
person_10    0.080273
person_16    0.077711
person_7     0.072588
person_11    0.066610
person_21    0.066610
person_8     0.058924
person_2     0.040137
person_19    0.036721
person_9     0.033305
person_17    0.023057
person_13    0.023057
person_18    0.022203
person_5     0.021349
person_20    0.020495
person_12    0.018787
person_4     0.016225
person_15    0.014518
person_3     0.012810
person_6     0.005978
person_1     0.005978
dtype: float64
